<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-image-classification/notebooks/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像分類（Image Classification）
入力した画像情報を「形」や「模様」、「色」など特徴量に分解し、入力情報から対象物を判定できるように対応付けを行う

## CNN（Convolutional Neural Network）
畳み込みニューラルネットワーク（CNNまたはConvNet）は、畳み込みを使用しているニューラルネットワークの総称

## ResNet
残差ニューラルネットワーク（ResNet）は、ウェイト層が層入力を参照して残差関数を学習する深層学習モデル

## VGG16
ImageNetと呼ばれる大規模画像データセットで学習された16層(畳み込み13層、フル結合3層)からなる畳み込みニューラルネットワーク(CNN)

## Efficient Net
複合スケーリング手法を用いて、ネットワークの幅、深さ、解像度を同時に最適化し、より少ないパラメータ数で高い精度を持つCNN

## 各手法の違い

In [10]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications

# CIFAR-10データセットの読み込み
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
# データの前処理（画素値を0-1の範囲に正規化）
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)  ##
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)  ##

class ImageClassificationModelBase:
  def __init__(self, x_train, y_train, x_test, y_test, filepath):
    self.filename = filepath.split('/')[-1]
    self.name = self.filename.split('.')[0]
    if os.path.exists(filepath):
      self.model = tf.keras.models.load_model(filepath)
    else:
      self._build_model()
      # optimizer: 最適化アルゴリズム, loss: 損失関数, metrics: 評価指標
      self.model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
      history = self.model.fit(
        x_train,
        y_train,
        batch_size=256,
        epochs=10,
        validation_data=(x_test, y_test))
      self.model.save(self.filename)

  def evaluate(self, x_test,  y_test):
    test_loss, test_acc = self.model.evaluate(x_test,  y_test, verbose=2)
    print(f'\n{self.name}: Test accuracy: {test_acc}')

  def predict(self, x, y):
    prediction = self.model.predict(x)
    print(f'\n{self.name}: Prediction: {prediction}, Actual: {y}')

  def _build_model(self):
    pass


class CnnModel(ImageClassificationModelBase):
  def _build_model(self):
    self.model = models.Sequential()
    self.model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    self.model.add(layers.MaxPooling2D((2, 2)))
    self.model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    self.model.add(layers.Flatten())
    self.model.add(layers.Dense(64, activation='relu'))
    self.model.add(layers.Dense(10, activation='softmax'))


class ResNetModel(ImageClassificationModelBase):
  def _build_model(self):
    # ResNet50モデルの読み込み（ImageNetで事前学習済み）
    # ImageNet:
    # 物体認識ソフトウェアの研究において、ソフトウェアの最適化や性能の評価等に
    # 用いるために設計された、大規模な画像データセット
    resnet_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

    # ResNet50の出力層をCIFAR-10用に変更
    # 事前学習済みのResNet50モデルに、CIFAR-10のクラス数に対応する全結合層を追加
    self.model = models.Sequential()
    self.model.add(resnet_model)
    self.model.add(layers.Flatten())
    self.model.add(layers.Dense(10, activation='softmax'))


class Vgg16Model(ImageClassificationModelBase):
  def _build_model(self):
    vgg16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    self.model = models.Sequential()
    self.model.add(vgg16_model)
    self.model.add(layers.Flatten())
    self.model.add(layers.Dense(10, activation='softmax'))


class EfficientNetModel(ImageClassificationModelBase):
  def _build_model(self):
    efficientnet_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    self.model = models.Sequential()
    self.model.add(efficientnet_model)
    self.model.add(layers.GlobalAveragePooling2D())  # GlobalAveragePooling2Dを追加
    self.model.add(layers.Dense(10, activation='softmax'))


# CNN
cnn_model = CnnModel(x_train, y_train, x_test, y_test, '/content/cnn.keras')
cnn_model.evaluate(x_test,  y_test)

# ResNet
resnet_model = ResNetModel(x_train, y_train, x_test, y_test, '/content/resnet.keras')
resnet_model.evaluate(x_test,  y_test)

# VGG16
vgg16_model = Vgg16Model(x_train, y_train, x_test, y_test, '/content/vgg16.keras')
vgg16_model.evaluate(x_test,  y_test)

# Efficient Net
effnet_model = EfficientNetModel(x_train, y_train, x_test, y_test, '/content/efficientnet.keras')
effnet_model.evaluate(x_test,  y_test)


313/313 - 2s - 5ms/step - accuracy: 0.6810 - loss: 0.9307

cnn: Test accuracy: 0.6809999942779541
313/313 - 8s - 25ms/step - accuracy: 0.7636 - loss: 0.9443

resnet: Test accuracy: 0.7635999917984009
313/313 - 4s - 12ms/step - accuracy: 0.8178 - loss: 0.6869

vgg16: Test accuracy: 0.817799985408783
313/313 - 12s - 38ms/step - accuracy: 0.0979 - loss: 3387.0740

efficientnet: Test accuracy: 0.09790000319480896
